In [2]:
import grammar as g
import utils as t
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import importlib
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision import datasets, transforms
from datetime import datetime 
import fitness_metrics

/home/alexserra98/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Grammar

In [3]:
gramm = g.Grammar('/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/cnn.grammar.txt')


### Phenotype

In [6]:
prova = gramm.initialise('features')

In [7]:
feat_layers = gramm.decode('features', prova)

In [8]:
feat_layers

'layer:conv num-filters:194 filter-shape:2 stride:3 padding:same act:relu bias:False'

In [9]:
layer_decoding = t.()
l_dict = layer_decoding.get_layers(feat_layers) 
l_dict

TypeError: Evaluator.__init__() missing 3 required positional arguments: 'fitness_metric', 'batch_size', and 'model'

In [10]:
feat_layers

'layer:pool-avg kernel-size:3 stride:1 padding:valid'

In [11]:
type(l_dict)

list

In [15]:
importlib.reload(t)

<module 'tmp' from '/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/tmp.py'>

In [14]:

assembler = t.Evaluator()
lay = assembler.assemble_network(l_dict,(28,28,3))

layer_type: pool-avg


In [15]:
type(lay)

torch.nn.modules.container.Sequential

### Assembling Layers

In [11]:
importlib.reload(t)

<module 'tmp' from '/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/tmp.py'>

In [10]:
gramm = g.Grammar('/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/cnn.grammar.txt')
init_max = {'features' : 5,'classification' : 1, 'learning' : 1}

#### feature module

In [11]:
my_module = t.Module('features',1,10,)
my_module1 = t.Module('classification',1,2)
my_module2 = t.Module('learning',1,2)



In [12]:
my_module.initialise(gramm,0.2,init_max)
my_module1.initialise(gramm,0.2,init_max)
my_module2.initialise(gramm,0.2,init_max)

In [16]:
my_module.layers[0]

{'features': [{'ge': 0, 'ga': {}}],
 'convolution': [{'ge': 0,
   'ga': {'num-filters': ('int', 32.0, 256.0, [36]),
    'filter-shape': ('int', 2.0, 5.0, [4]),
    'stride': ('int', 1.0, 3.0, [2])}}],
 'padding': [{'ge': 1, 'ga': {}}],
 'activation-function': [{'ge': 2, 'ga': {}}],
 'bias': [{'ge': 1, 'ga': {}}]}

In [13]:
my_layers_decoded = []
my_layers_decoded_1 = []
my_layers_decoded_2 = []

for i in my_module.layers:
    my_layers_decoded.append(gramm.decode('features', i))

for i in my_module1.layers:
    my_layers_decoded_1.append(gramm.decode('classification', i))

for i in my_module2.layers:
    my_layers_decoded_2.append(gramm.decode('learning', i))

pheno_class =' '.join(my_layers_decoded_1)
pheno_feat =' '.join(my_layers_decoded)
pheno_learn = ' '.join(my_layers_decoded_2)

In [14]:
pheno_feat

'layer:conv num-filters:36 filter-shape:4 stride:2 padding:valid act:sigmoid bias:False layer:conv num-filters:248 filter-shape:4 stride:1 padding:same act:relu bias:False layer:pool-max kernel-size:2 stride:3 padding:same layer:pool-avg kernel-size:4 stride:1 padding:same layer:pool-max kernel-size:2 stride:3 padding:same'

In [17]:
encoder = t.Net_encoding(5, 1,3,2, (32,32,3), pheno_feat, pheno_class, pheno_learn)

In [19]:
dict = encoder.get_layers(pheno_feat)

In [21]:
dict[0][0]

'conv'

In [8]:
learn_param = encoder.get_learning()

In [9]:
mymodel = encoder.assemble_model()

layer_type: conv
layer_type: conv
layer_type: conv
layer_type: pool-avg
layer_type: conv
layer_type: fc


In [17]:
summary(mymodel,(3,32,32),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 170, 14, 14]          12,750
           Sigmoid-2          [-1, 170, 14, 14]               0
            Conv2d-3          [-1, 162, 14, 14]         440,802
           Sigmoid-4          [-1, 162, 14, 14]               0
            Conv2d-5          [-1, 243, 14, 14]         629,856
              Tanh-6          [-1, 243, 14, 14]               0
         AvgPool2d-7            [-1, 243, 5, 5]               0
            Conv2d-8            [-1, 102, 1, 1]         396,678
              ReLU-9            [-1, 102, 1, 1]               0
           Linear-10                 [-1, 1241]         127,823
             Tanh-11                 [-1, 1241]               0
           Linear-12                   [-1, 10]          12,420
Total params: 1,620,329
Trainable params: 1,620,329
Non-trainable params: 0
---------------------------

/home/alexserra98/miniconda3/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/conv.py:443: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755897462/work/aten/src/ATen/native/Convolution.cpp:744.)
  return F.conv2d(input, weight, bias, self.stride,


In [18]:
optimizer = encoder.assemble_optimiser(mymodel)

In [19]:
optimizer

RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.05256362464710246
    momentum: 0
    weight_decay: 0
)

In [152]:
mymodel

genNN(
  (fe): Sequential(
    (0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout2d(p=0.08870683997615111, inplace=False)
    (2): Conv2d(3, 124, kernel_size=(5, 5), stride=(1, 1), padding=same, bias=False)
    (3): ReLU()
    (4): AvgPool2d(kernel_size=(4, 4), stride=3, padding=0)
    (5): Conv2d(124, 229, kernel_size=(2, 2), stride=(2, 2), padding=valid, bias=False)
    (6): Sigmoid()
    (7): Conv2d(229, 173, kernel_size=(4, 4), stride=(1, 1), padding=same)
    (8): Sigmoid()
    (9): MaxPool2d(kernel_size=(2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
    (10): BatchNorm2d(173, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (c): Sequential(
    (0): Linear(in_features=173, out_features=446, bias=False)
    (1): Tanh()
  )
  (last): Sequential(
    (0): Linear(in_features=446, out_features=10, bias=True)
  )
)

In [20]:
# parameters
RANDOM_SEED = 42
LEARNING_RATE = 0.001
BATCH_SIZE = 32
N_EPOCHS = 15

IMG_SIZE = 32
N_CLASSES = 10

In [21]:

def get_accuracy(model, data_loader, device):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''
    
    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            X = X.to(device)
            y_true = y_true.to(device)

            _, y_prob = model(X)
            _, predicted_labels = torch.max(y_prob, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n

def plot_losses(train_losses, valid_losses):
    '''
    Function for plotting training and validation losses
    '''
    
    # temporarily change the style of the plots to seaborn 
    plt.style.use('seaborn')

    train_losses = np.array(train_losses) 
    valid_losses = np.array(valid_losses)

    fig, ax = plt.subplots(figsize = (8, 4.5))

    ax.plot(train_losses, color='blue', label='Training loss') 
    ax.plot(valid_losses, color='red', label='Validation loss')
    ax.set(title="Loss over epochs", 
            xlabel='Epoch',
            ylabel='Loss') 
    ax.legend()
    fig.show()
    
    # change the plot style to default
    plt.style.use('default')
    
def train(train_loader, model, criterion, optimizer, device):
    '''
    Function for the training step of the training loop
    '''

    model.train()
    running_loss = 0
    
    for X, y_true in train_loader:

        optimizer.zero_grad()
        
        X = X.to(device)
        y_true = y_true.to(device)
    
        # Forward pass
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

        # Backward pass
        loss.backward()
        optimizer.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss

def validate(valid_loader, model, criterion, device):
    '''
    Function for the validation step of the training loop
    '''
   
    model.eval()
    running_loss = 0
    
    for X, y_true in valid_loader:
    
        X = X.to(device)
        y_true = y_true.to(device)

        # Forward pass and record loss
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
        
    return model, epoch_loss

def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, device, print_every=1):
    '''
    Function defining the entire training loop
    '''
    
    # set objects for storing metrics
    best_loss = 1e10
    train_losses = []
    valid_losses = []
 
    # Train model
    for epoch in range(0, epochs):

        # training
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
        train_losses.append(train_loss)

        # validation
        with torch.no_grad():
            model, valid_loss = validate(valid_loader, model, criterion, device)
            valid_losses.append(valid_loss)

        if epoch % print_every == (print_every - 1):
            
            train_acc = get_accuracy(model, train_loader, device=device)
            valid_acc = get_accuracy(model, valid_loader, device=device)
                
            print(f'{datetime.now().time().replace(microsecond=0)} --- '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}\t'
                  f'Valid accuracy: {100 * valid_acc:.2f}')

    plot_losses(train_losses, valid_losses)
    
    return model, optimizer, (train_losses, valid_losses)

# define transforms
# transforms.ToTensor() automatically scales the images to [0,1] range
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()])

# download and create datasets
train_dataset = datasets.CIFAR10(root='cifar_data', 
                               train=True, 
                               transform=transforms,
                               download=True)

valid_dataset = datasets.CIFAR10(root='cifar_data', 
                               train=False, 
                               transform=transforms)

# define the data loaders
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=False)

Files already downloaded and verified


In [22]:
model = mymodel
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()


In [15]:
DEVICE = 'cpu'
model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, N_EPOCHS, DEVICE)


09:47:49 --- Epoch: 0	Train loss: 1.6836	Valid loss: 1.4797	Train accuracy: 46.78	Valid accuracy: 46.45
09:49:09 --- Epoch: 1	Train loss: 1.4204	Valid loss: 1.3766	Train accuracy: 52.66	Valid accuracy: 51.32
09:50:37 --- Epoch: 2	Train loss: 1.3105	Valid loss: 1.2896	Train accuracy: 56.75	Valid accuracy: 53.78


KeyboardInterrupt: 

In [34]:
importlib.reload(t)

<module 'tmp' from '/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/tmp.py'>

In [33]:
importlib.reload(fitness_metrics)

<module 'fitness_metrics' from '/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/fitness_metrics.py'>

In [35]:
eval = t.Evaluator(fitness_metrics.accuracy,1,model)


In [36]:
eval.train(0,"",train_loader,valid_loader, optimizer, epochs=3)

evaluating: 100%|██████████| 313/313 [00:04<00:00, 72.52it/s]


16:21:05 --- Epoch: 0	Train loss: 2.3032	Valid loss: 2.3028	Train accuracy: 10.00	Valid accuracy: 10.00


training:  70%|███████   | 1096/1563 [00:40<00:17, 26.80it/s]


KeyboardInterrupt: 

In [ ]:
filename = os.path.join(folder_save, "model.pt")

In [55]:
type(model)

tmp.genNN

In [65]:
importlib.reload(t)

<module 'tmp' from '/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/tmp.py'>

In [66]:
ind = t.Individual([('features',1,10),('classification',1,2)],['learning'],0)

In [67]:
ind.initialise(gramm,0.2,init_max)

self.macro: [{'learning': [{'ge': 2, 'ga': {'batch_size': ('int', 50.0, 500.0, [391])}}], 'adam': [{'ge': 0, 'ga': {'lr': ('float', 0.0001, 0.1, [0.06883574665862617]), 'beta1': ('float', 0.5, 1.0, [0.5806125431159568]), 'beta2': ('float', 0.5, 1.0, [0.7261197681079284]), 'decay': ('float', 1e-06, 0.001, [0.0009131089004599922])}}], 'early-stop': [{'ge': 0, 'ga': {'early_stop': ('int', 5.0, 20.0, [14])}}]}]


In [69]:
ind.decode(gramm)

'layer:batch-norm layer:conv num-filters:71 filter-shape:5 stride:2 padding:same act:relu bias:False layer:conv num-filters:243 filter-shape:5 stride:2 padding:same act:tanh bias:False layer:batch-norm layer:conv num-filters:63 filter-shape:3 stride:2 padding:same act:tanh bias:False layer:fc act:sigmoid num-units:584 bias:False learning:adam lr:0.06883574665862617 beta1:0.5806125431159568 beta2:0.7261197681079284 decay:0.0009131089004599922 early_stop:14 batch_size:391 epochs:400'

In [68]:
ind.macro

[{'learning': [{'ge': 2, 'ga': {'batch_size': ('int', 50.0, 500.0, [391])}}],
  'adam': [{'ge': 0,
    'ga': {'lr': ('float', 0.0001, 0.1, [0.06883574665862617]),
     'beta1': ('float', 0.5, 1.0, [0.5806125431159568]),
     'beta2': ('float', 0.5, 1.0, [0.7261197681079284]),
     'decay': ('float', 1e-06, 0.001, [0.0009131089004599922])}}],
  'early-stop': [{'ge': 0, 'ga': {'early_stop': ('int', 5.0, 20.0, [14])}}]}]